In [39]:
from openai import OpenAI
from pydantic import BaseModel


class Row(BaseModel):
    zip_code: str
    city: str
    state: str
    avg_temp_f: float
    population: int
    median_income: int
    known_for: str

class Rows(BaseModel):
    rows: list[Row]

def generate_dataset_rows(location: str = "United States", count: int = 5, year: int = 2022):
    client = OpenAI()

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Please generate {count} rows of data for random zip codes in {location} for the year {year}."},
        ],
        response_format=Rows
        
    )

    return completion.choices[0].message.parsed.model_dump()['rows']

rows = generate_dataset_rows()

In [40]:
import weave


@weave.op
def check_concrete_fields(city: str, state: str, model_output: dict):
    return {
        "city_match": city == model_output["city"],
        "state_match": state == model_output["state"],
    }

@weave.op
def check_value_fields(avg_temp_f: float, population: int, median_income: int, model_output: dict):
    return {
        "avg_temp_f_err": abs(avg_temp_f - model_output["avg_temp_f"]) / avg_temp_f,
        "population_err": abs(population - model_output["population"]) / population,
        "median_income_err": abs(median_income - model_output["median_income"]) / median_income,
    }

@weave.op
def check_subjective_fields(zip_code: str, known_for: str, model_output: dict):
    client = OpenAI()

    class Response(BaseModel):
        correct_known_for: bool

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"My student was asked what the zip code {zip_code} is best known best for. The right answer is '{known_for}', and they said '{model_output['known_for']}'. Is their answer correct?"},
        ],
        response_format=Response
    )

    return completion.choices[0].message.parsed.model_dump()


evaluation = weave.Evaluation(
    name="United States - 2022 - Concrete Fields",
    dataset=rows,
    scorers=[
        check_concrete_fields,
        check_value_fields,
        check_subjective_fields,
    ]
)

In [41]:
weave.init("leaderboard-demo-1")

@weave.op
def baseline_model(zip_code: str):
    return {
        "city": "New York",
        "state": "NY",
        "avg_temp_f": 50.0,
        "population": 1000000,
        "median_income": 100000,
        "known_for": "The Big Apple",
    }

await evaluation.evaluate(baseline_model)

Evaluated 1 of 5 examples

Evaluated 2 of 5 examples

Evaluated 3 of 5 examples

Evaluated 4 of 5 examples

Evaluated 5 of 5 examples

Evaluation summary
{
    'model_output': {
        'avg_temp_f': {'mean': 50.0},
        'population': {'mean': 1000000.0},
        'median_income': {'mean': 100000.0}
    },
    'check_concrete_fields': {
        'city_match': {'true_count': 1, 'true_fraction': 0.2},
        'state_match': {'true_count': 1, 'true_fraction': 0.2}
    },
    'check_value_fields': {
        'median_income_err': {'mean': 0.6228438228438228},
        'avg_temp_f_err': {'mean': 0.16338317484024056},
        'population_err': {'mean': 0.6866822079727841}
    },
    'check_subjective_fields': {'correct_known_for': {'true_count': 0, 'true_fraction': 0.0}},
    'model_latency': {'mean': 0.005936479568481446}
}

🍩 https://wandb.ai/timssweeney/leaderboard-demo-1/r/call/0192b84c-550a-7701-8ca5-47609c8a5892


{'model_output': {'avg_temp_f': {'mean': 50.0},
  'population': {'mean': 1000000.0},
  'median_income': {'mean': 100000.0}},
 'check_concrete_fields': {'city_match': {'true_count': 1,
   'true_fraction': 0.2},
  'state_match': {'true_count': 1, 'true_fraction': 0.2}},
 'check_value_fields': {'median_income_err': {'mean': 0.6228438228438228},
  'avg_temp_f_err': {'mean': 0.16338317484024056},
  'population_err': {'mean': 0.6866822079727841}},
 'check_subjective_fields': {'correct_known_for': {'true_count': 0,
   'true_fraction': 0.0}},
 'model_latency': {'mean': 0.005936479568481446}}

In [42]:
weave.init("leaderboard-demo-1")

@weave.op
def gpt_4o_mini_no_context(zip_code: str):
    client = OpenAI()

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": f"""Zip code {zip_code}"""}],
        response_format=Row
    )

    return completion.choices[0].message.parsed.model_dump()

await evaluation.evaluate(baseline_model)

Evaluated 1 of 5 examples

Evaluated 2 of 5 examples

Evaluated 3 of 5 examples

Evaluated 4 of 5 examples

Evaluated 5 of 5 examples

Evaluation summary
{
    'model_output': {
        'avg_temp_f': {'mean': 50.0},
        'population': {'mean': 1000000.0},
        'median_income': {'mean': 100000.0}
    },
    'check_concrete_fields': {
        'city_match': {'true_count': 1, 'true_fraction': 0.2},
        'state_match': {'true_count': 1, 'true_fraction': 0.2}
    },
    'check_value_fields': {
        'median_income_err': {'mean': 0.6228438228438228},
        'avg_temp_f_err': {'mean': 0.16338317484024056},
        'population_err': {'mean': 0.6866822079727841}
    },
    'check_subjective_fields': {'correct_known_for': {'true_count': 0, 'true_fraction': 0.0}},
    'model_latency': {'mean': 0.00630335807800293}
}

🍩 https://wandb.ai/timssweeney/leaderboard-demo-1/r/call/0192b84c-6a1b-7651-9a71-f86d8eb6d4da


{'model_output': {'avg_temp_f': {'mean': 50.0},
  'population': {'mean': 1000000.0},
  'median_income': {'mean': 100000.0}},
 'check_concrete_fields': {'city_match': {'true_count': 1,
   'true_fraction': 0.2},
  'state_match': {'true_count': 1, 'true_fraction': 0.2}},
 'check_value_fields': {'median_income_err': {'mean': 0.6228438228438228},
  'avg_temp_f_err': {'mean': 0.16338317484024056},
  'population_err': {'mean': 0.6866822079727841}},
 'check_subjective_fields': {'correct_known_for': {'true_count': 0,
   'true_fraction': 0.0}},
 'model_latency': {'mean': 0.00630335807800293}}

In [43]:
weave.init("leaderboard-demo-1")

@weave.op
def gpt_4o_mini_with_context(zip_code: str):
    client = OpenAI()

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": f"""Please answer the following questions about the zip code {zip_code}:
                   1. What is the city?
                   2. What is the state?
                   3. What is the average temperature in Fahrenheit?
                   4. What is the population?
                   5. What is the median income?
                   6. What is the most well known thing about this zip code?
                   """}],
        response_format=Row
    )

    return completion.choices[0].message.parsed.model_dump()

await evaluation.evaluate(baseline_model)

Evaluated 1 of 5 examples

Evaluated 2 of 5 examples

Evaluated 3 of 5 examples

Evaluated 4 of 5 examples

Evaluated 5 of 5 examples

Evaluation summary
{
    'model_output': {
        'avg_temp_f': {'mean': 50.0},
        'population': {'mean': 1000000.0},
        'median_income': {'mean': 100000.0}
    },
    'check_concrete_fields': {
        'city_match': {'true_count': 1, 'true_fraction': 0.2},
        'state_match': {'true_count': 1, 'true_fraction': 0.2}
    },
    'check_value_fields': {
        'median_income_err': {'mean': 0.6228438228438228},
        'avg_temp_f_err': {'mean': 0.16338317484024056},
        'population_err': {'mean': 0.6866822079727841}
    },
    'check_subjective_fields': {'correct_known_for': {'true_count': 0, 'true_fraction': 0.0}},
    'model_latency': {'mean': 0.009626293182373047}
}

🍩 https://wandb.ai/timssweeney/leaderboard-demo-1/r/call/0192b84c-87c0-7d92-b470-75a25d3a172d


{'model_output': {'avg_temp_f': {'mean': 50.0},
  'population': {'mean': 1000000.0},
  'median_income': {'mean': 100000.0}},
 'check_concrete_fields': {'city_match': {'true_count': 1,
   'true_fraction': 0.2},
  'state_match': {'true_count': 1, 'true_fraction': 0.2}},
 'check_value_fields': {'median_income_err': {'mean': 0.6228438228438228},
  'avg_temp_f_err': {'mean': 0.16338317484024056},
  'population_err': {'mean': 0.6866822079727841}},
 'check_subjective_fields': {'correct_known_for': {'true_count': 0,
   'true_fraction': 0.0}},
 'model_latency': {'mean': 0.009626293182373047}}

In [49]:
scorers = [
    check_concrete_fields,
    check_value_fields,
    check_subjective_fields,
]
evaluations = [
    weave.Evaluation(
        name="United States - 2022",
        dataset=weave.Dataset(name="United States - 2022", rows=generate_dataset_rows("United States", 5, 2022)),
        scorers=scorers,
    ),
    weave.Evaluation(
        name="California - 2022",
        dataset=weave.Dataset(name="California - 2022", rows=generate_dataset_rows("California", 5, 2022)),
        scorers=scorers,
    ),
    # weave.Evaluation(
    #     name="United States - 2000",
    #     dataset=weave.Dataset(name="United States - 2000", rows=generate_dataset_rows("United States", 5, 2000)),
    #     scorers=scorers,
    # ),
]
models = [
    baseline_model,
    gpt_4o_mini_no_context,
    gpt_4o_mini_with_context,
]

for evaluation in evaluations:
    for model in models:
        await evaluation.evaluate(model, __weave={"display_name": evaluation.name + ":" + model.__name__})

Evaluated 1 of 5 examples

Evaluated 2 of 5 examples

Evaluated 3 of 5 examples

Evaluated 4 of 5 examples

Evaluated 5 of 5 examples

...

Evaluated 3 of 5 examples

Evaluated 4 of 5 examples

Evaluated 5 of 5 examples

Evaluation summary
{
    'model_output': {
        'avg_temp_f': {'mean': 60.9},
        'population': {'mean': 75024.8},
        'median_income': {'mean': 84372.6}
    },
    'check_concrete_fields': {
        'city_match': {'true_count': 5, 'true_fraction': 1.0},
        'state_match': {'true_count': 2, 'true_fraction': 0.4}
    },
    'check_value_fields': {
        'median_income_err': {'mean': 0.3543283982836134},
        'avg_temp_f_err': {'mean': 0.015047371269703083},
        'population_err': {'mean': 70.28742213711891}
    },
    'check_subjective_fields': {'correct_known_for': {'true_count': 5, 'true_fraction': 1.0}},
    'model_latency': {'mean': 2.426439666748047}
}

🍩 https://wandb.ai/timssweeney/leaderboard-demo-1/r/call/0192b853-57cf-7cd0-a363-d42b22f5ec9c
